#CLIMATE DATA
This code downloads climate information in a certain area. Downloaded data includes daily maximum and minimum temperature and cumulative daily precipitation. The data are processed and made available in the [Google Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY#description) by the Copernicus Climate Change Service.
The downloaded data come from the ERA 5 project, which is the 5th generation of ECMWF atmospheric reanalysis of global climate. Reanalysis combines model data with observations from across the world into a globally complete and consistent dataset.
Maximum and minimum temperatures are evaluated 2 meters above the ground and were calculated from ERA 5 temperature data.
The output of the code are georeferenced maps for the Po river basin area. The output reference system is ESRI:50012. The output pixel size is 250x250 m which is consistent with the SWB input maps. The resolution of the climate data provided does not match the input grid, so the value assigned to each pixel is sampled from the downloaded map which has a resolution of 0.25 arc degrees.



#Libraries Installation
The first library installed is rasterio. This library allowed to elaborate save and load the raster files. it will be used to elaborate the maps loaded form Google Earth Engine. The second library is fiona that allow to clip the downloaded raster file with a mask (a shape file of the selected area).

In [ ]:
pip install rasterio

     |████████████████████████████████| 19.3MB 55.7MB/s 


In [ ]:
pip install fiona

     |████████████████████████████████| 15.4MB 201kB/s 


#Acces to google earth engine and import all the libraries
This section imports all the libraries that are used in the following code to download and process the data. This section also initializes Earth Engine. Follow the displayed instruction to log in. It is necessary a Google Earth Engine account to precede with the log in and to run the following code.

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=3xjUpDm6TUlv_8_s5uCYiem49T9qTz8BBy9zA8SE6rM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhELt0TMIZXWznjTfXFXrbqUcCZzHZiF5VuF5NNhzuK_sm4JwC1AtU

Successfully saved authorization token.


In [ ]:
import requests
import zipfile
import numpy as np
import calendar
import fiona
import rasterio.mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import calendar
import os
import ipywidgets as widgets
import smtplib
from google.colab import files
import zlib
from osgeo import gdal


#Slect period and area

In this section it is possible to select the area of interest and the dates between which the daily data are to be downloaded.
Warning! An area larger than the SWB grid area must be selected. The coordinates of the area must be provided according to the WGS 84 reference system, without considering the RS of the output.


In [ ]:
yearStart = 2019
yearEnd = 2019
monthStart = 1
monthEnd = 12


left = 6
down = 43
right =14
up = 47

#Download maximum temperature data

The download section is divided in three subsections. The first section downloads and pre-process the Maximum temperature data.
The code first downloads the raster from the data catalog, the data is downloaded for the area selected in the previous section and is provided as a compressed folder containing a raster file.
The downloaded zipped file is extracted, reprojected, resampled (to the grid resolution) and clipped coherently with the SWB input grid box.
The code requires a shape file of the area of interest (Po river basin) referenced to the SR of the output file.
The output reference system is defined in the code as a string in the dst_crs variable.
Finally, the output file is converted to arc ASCII (.asc) format, is compressed, and is saved in a compressed folder that contains all data from the same year.

When all daily files for a given year are zipped into the folder, an automatic download of the compressed folder begins.

When the code has completed the creation of all the compressed folders, an email can be automatically sent to the user's email. Enter your e-mail address in the code line (substitute your@email.com).





In [ ]:
w = widgets.IntProgress(
    value=0,
    min=0,
    max=(yearEnd-yearStart+1)*12,
    description='Loading:',
    bar_style='success',
    orientation='horizontal')
display(w)
 
for year in list(range(yearStart, yearEnd + 1)):
 zip_name = 'tmax_%04d.zip' % (year)
 zipObj = zipfile.ZipFile(zip_name, 'w',zipfile.ZIP_DEFLATED )
 for month in list(range(monthStart, monthEnd + 1)):
  numberOfDays = calendar.monthrange(year, month)[1]
  for day in  list(range(1, numberOfDays+1)):
    targetFile = "tmax_%04d_%02d_%02d.tif" % (year, month, day)
    requestDate = '%04d-%02d-%02d' % (year, month, day)

    if (day+1)>numberOfDays and  month == 12:
      dayAfter = '%04d-%02d-%02d'  % (year+1, 1, 1)
    elif (day+1)>numberOfDays:
      dayAfter = '%04d-%02d-%02d'  % (year, month+1, 1)     
    else:
      dayAfter = '%04d-%02d-%02d'  % (year, month, (day+1))    
    
    # Import the MODIS land cover collection.
    dataset = ee.ImageCollection("ECMWF/ERA5/DAILY").filter(ee.Filter.date(requestDate, dayAfter)).first()
    aoi = ee.Geometry.Rectangle(left,down,right,up)
    img = dataset.clip(aoi)

    url = img.getDownloadURL({
      'bands': 'maximum_2m_air_temperature',
      'min': 250,
      'max': 320,})

    filename = 'Myfile.zip'

    r = requests.get(url, stream=True)
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=None):
            fd.write(chunk)
    z = zipfile.ZipFile(filename)
    z.extractall()

    dst_crs = 'ESRI:54012'

    FileName = '%04d%02d%02d.maximum_2m_air_temperature.tif'  % (year, month, (day))

    with rasterio.open(FileName) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)#, resolution=([250,250]))
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open('reprojected.tif', 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
 

    with rasterio.open('reprojected.tif') as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, left=540000.0, bottom=5300000.0, right=1200000.0, top=5800000.0, resolution=([250,250]))
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open('reproj_cut.tif', 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
            
    with fiona.open("box.shp", "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rasterio.open("reproj_cut.tif") as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(targetFile, "w", **out_meta) as dest:
        dest.write((((out_image-273.15)*1.8)+32)) #converted in °F

    targetFile_ascii =  "tmax_%04d_%02d_%02d.asc" % (year, month, day)
    targetFile_prj =  "tmax_%04d_%02d_%02d.prj" % (year, month, day)
    #Open existing dataset
    src_ds = gdal.Open( targetFile )
    #Open output format driver, see gdal_translate --formats for list
    format = "AAIGrid"
    driver = gdal.GetDriverByName( format )

    #Output to new format
    dst_ds = driver.CreateCopy( targetFile_ascii, src_ds, 0 )

    #Properly close the datasets to flush to disk
    dst_ds = None
    src_ds = None

    zipObj.write(targetFile_ascii)
    os.remove(targetFile)
    os.remove(targetFile_ascii)
    os.remove(targetFile_ascii+'.aux.xml')
    os.remove(targetFile_prj)
    os.remove(FileName)
  w.value += 1
 # close the Zip File
 zipObj.close() 
 files.download(zip_name) 
  
content = ('Ei dummy, the calculation in colab is finished! download them before you lose everything!')
mail = smtplib.SMTP('smtp.gmail.com',587)

mail.ehlo()

mail.starttls()

mail.login('meteodatapython@gmail.com','pythoncode1')

mail.sendmail('meteodatapython@gmail.com','your@email.com',content) 

mail.close()

print("Sent")

IntProgress(value=0, bar_style='success', description='Loading:', max=12)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sent


#Download minimum temperature data
In this section are performed the same operation of the previous section for the minimum temperature data.

In [ ]:
w = widgets.IntProgress(
    value=0,
    min=0,
    max=(yearEnd-yearStart+1)*12,
    description='Loading:',
    bar_style='success',
    orientation='horizontal')
display(w)
 
for year in list(range(yearStart, yearEnd + 1)):
 zip_name = 'tmin_%04d.zip' % (year)	
 zipObj = zipfile.ZipFile(zip_name, 'w',zipfile.ZIP_DEFLATED )
 for month in list(range(monthStart, monthEnd + 1)):
  numberOfDays = calendar.monthrange(year, month)[1]
  for day in  list(range(1, numberOfDays+1)):
    targetFile = "tmin_%04d_%02d_%02d.tif" % (year, month, day)
    requestDate = '%04d-%02d-%02d' % (year, month, day)

    if (day+1)>numberOfDays and  month == 12:
      dayAfter = '%04d-%02d-%02d'  % (year+1, 1, 1)
    elif (day+1)>numberOfDays:
      dayAfter = '%04d-%02d-%02d'  % (year, month+1, 1)     
    else:
      dayAfter = '%04d-%02d-%02d'  % (year, month, (day+1))    
    
    # Import the MODIS land cover collection.
    dataset = ee.ImageCollection("ECMWF/ERA5/DAILY").filter(ee.Filter.date(requestDate, dayAfter)).first()
    aoi = ee.Geometry.Rectangle(left,down,right,up)
    img = dataset.clip(aoi)

    url = img.getDownloadURL({
      'bands': 'minimum_2m_air_temperature',
      'min': 250,
      'max': 320,})

    filename = 'Myfile.zip'

    r = requests.get(url, stream=True)
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=None):
            fd.write(chunk)
    z = zipfile.ZipFile(filename)
    z.extractall()

    dst_crs = 'ESRI:54012'

    FileName = '%04d%02d%02d.minimum_2m_air_temperature.tif'  % (year, month, (day))

    with rasterio.open(FileName) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)#, resolution=([250,250]))
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open('reprojected.tif', 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
 

    with rasterio.open('reprojected.tif') as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, left=540000.0, bottom=5300000.0, right=1200000.0, top=5800000.0, resolution=([250,250]))
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open('reproj_cut.tif', 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
            
    with fiona.open("box.shp", "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rasterio.open("reproj_cut.tif") as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(targetFile, "w", **out_meta) as dest:
        dest.write((((out_image-273.15)*1.8)+32)) #converted in °F
	
    targetFile_ascii =  "tmin_%04d_%02d_%02d.asc" % (year, month, day)	
    targetFile_prj =  "tmin_%04d_%02d_%02d.prj" % (year, month, day)	
    #Open existing dataset	
    src_ds = gdal.Open( targetFile )	
    #Open output format driver, see gdal_translate --formats for list	
    format = "AAIGrid"	
    driver = gdal.GetDriverByName( format )	
    #Output to new format	
    dst_ds = driver.CreateCopy( targetFile_ascii, src_ds, 0 )	
    #Properly close the datasets to flush to disk	
    dst_ds = None	
    src_ds = None	
    zipObj.write(targetFile_ascii)	
    os.remove(targetFile)	
    os.remove(targetFile_ascii)	
    os.remove(targetFile_ascii+'.aux.xml')	
    os.remove(targetFile_prj)    
    os.remove(FileName)
  w.value += 1
 # close the Zip File	
 zipObj.close() 	
 files.download(zip_name) 

content = ('Ei dummy, the calculation in colab is finished! download them before you lose everything!')
 
mail = smtplib.SMTP('smtp.gmail.com',587)

mail.ehlo()

mail.starttls()

mail.login('meteodatapython@gmail.com','pythoncode1')

mail.sendmail('meteodatapython@gmail.com','your@email.com',content) 

mail.close()

print("Sent")

IntProgress(value=0, bar_style='success', description='Loading:', max=12)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sent


#Download precipitation data

In this section are performed the same operation of the previous section for the precipitation data.



In [ ]:
w = widgets.IntProgress(
    value=0,
    min=0,
    max=(yearEnd-yearStart+1)*12,
    description='Loading:',
    bar_style='success',
    orientation='horizontal')
display(w)
 
for year in list(range(yearStart, yearEnd + 1)):
 zip_name = 'precip_%04d.zip' % (year)	
 zipObj = zipfile.ZipFile(zip_name, 'w',zipfile.ZIP_DEFLATED )
 for month in list(range(monthStart, monthEnd + 1)):
  numberOfDays = calendar.monthrange(year, month)[1]
  for day in  list(range(1, numberOfDays+1)):
    targetFile = "precip_%04d_%02d_%02d.tif" % (year, month, day)
    requestDate = '%04d-%02d-%02d' % (year, month, day)

    if (day+1)>numberOfDays and  month == 12:
      dayAfter = '%04d-%02d-%02d'  % (year+1, 1, 1)
    elif (day+1)>numberOfDays:
      dayAfter = '%04d-%02d-%02d'  % (year, month+1, 1)     
    else:
      dayAfter = '%04d-%02d-%02d'  % (year, month, (day+1))    
    
    # Import the MODIS land cover collection.
    dataset = ee.ImageCollection("ECMWF/ERA5/DAILY").filter(ee.Filter.date(requestDate, dayAfter)).first()
    aoi = ee.Geometry.Rectangle(left,down,right,up)
    img = dataset.clip(aoi)

    url = img.getDownloadURL({
      'bands': 'total_precipitation',
      'min': 250,
      'max': 320,})

    filename = 'Myfile.zip'

    r = requests.get(url, stream=True)
    with open(filename, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=None):
            fd.write(chunk)
    z = zipfile.ZipFile(filename)
    z.extractall()

    dst_crs = 'ESRI:54012'

    FileName = '%04d%02d%02d.total_precipitation.tif'  % (year, month, (day))

    with rasterio.open(FileName) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)#, resolution=([250,250]))
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open('reprojected.tif', 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
 

    with rasterio.open('reprojected.tif') as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, left=540000.0, bottom=5300000.0, right=1200000.0, top=5800000.0, resolution=([250,250]))
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open('reproj_cut.tif', 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)
            
    with fiona.open("box.shp", "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]

    with rasterio.open("reproj_cut.tif") as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_meta = src.meta

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(targetFile, "w", **out_meta) as dest:
        dest.write((out_image*39.3701)) # converted in inches
	
    targetFile_ascii =  "precip_%04d_%02d_%02d.asc" % (year, month, day)	
    targetFile_prj =  "precip_%04d_%02d_%02d.prj" % (year, month, day)	
    #Open existing dataset	
    src_ds = gdal.Open( targetFile )	
    #Open output format driver, see gdal_translate --formats for list	
    format = "AAIGrid"	
    driver = gdal.GetDriverByName( format )	
    #Output to new format	
    dst_ds = driver.CreateCopy( targetFile_ascii, src_ds, 0 )	
    #Properly close the datasets to flush to disk	
    dst_ds = None	
    src_ds = None	
    zipObj.write(targetFile_ascii)	
    os.remove(targetFile)	
    os.remove(targetFile_ascii)	
    os.remove(targetFile_ascii+'.aux.xml')	
    os.remove(targetFile_prj)
    os.remove(FileName)
  w.value += 1
 # close the Zip File	
 zipObj.close() 	
 files.download(zip_name) 

content = ('Ei dummy, the calculation in colab is finished! download them before you lose everything!')
mail = smtplib.SMTP('smtp.gmail.com',587)

mail.ehlo()

mail.starttls()

mail.login('meteodatapython@gmail.com','pythoncode1')

mail.sendmail('meteodatapython@gmail.com','your@email.com',content) 

mail.close()

print("Sent")

IntProgress(value=0, bar_style='success', description='Loading:', max=12)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Sent


#Referencies

Copernicus Climate Change Service (C3S) (2017): ERA5: Fifth generation of ECMWF atmospheric reanalyses of the global climate. Copernicus Climate Change Service Climate Data Store (CDS), (date of access), https://cds.climate.copernicus.eu/cdsapp#!/home